In [1]:
# !git clone https://github.com/sobiodum/quantumai3.git
# !pip3 install --no-cache-dir -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import ray
from configs.load_specific_data import pharma_basket
from env.multi_agent.worker_hrl import Worker
from env.multi_agent.hrl import HRL
from ray.rllib.utils import check_env
from ray.train.rl import RLTrainer
from ray.air.config import RunConfig, ScalingConfig
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.rllib.algorithms.bc.bc import BC
from ray.rllib.algorithms import a2c
from ray.tune.registry import register_env
import gymnasium
ray.init(_temp_dir='/Volumes/SSD980/ray')
import warnings
warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import os
os.environ['PYTHONWARNINGS'] = 'ignore'

2023-09-07 19:17:53,625	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-09-07 19:17:58,534	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


In [3]:
train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()
# train_df = train_df[train_df["tic"] == "PFE.US"]
# validate_df = validate_df[validate_df["tic"] == "PFE.US"]


In [4]:
train_df.to_csv("train.csv")

In [ ]:

manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [6]:
import os 
manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hierarch_env", env_creator)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 

first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 
manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }


trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 100},local_dir="/Volumes/SSD980/ray/results/"),
    scaling_config=ScalingConfig(
        num_workers=1,
        use_gpu=False,
    ),
 
    algorithm=a2c.A2C,
    config={
        "multiagent": {
            "policies": {
                "worker_policy": worker_policy_spec,
                "manager_policy": manager_policy_spec,
            },
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env": "hierarch_env",
        "env_config": hrl_config,
        "framework": "tf2",
        "evaluation_num_workers": 5,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

2023-09-06 22:55:27,918	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=69670) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=69670) 2023-09-06 22:55:35,610	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=69670) 2023-09-06 22:55:35,610	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=69670) 2023-09-06 22:55:35,610	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=69688) DeprecationWarning

(RolloutWorker pid=69707) =========HRL is done=============
(RolloutWorker pid=69707) HRL is done
(RolloutWorker pid=69707) day: 6304, episode: 2
(RolloutWorker pid=69707) Total Cash Transfers: 56
(RolloutWorker pid=69707) total_portfolio_trades: 48700.0
(RolloutWorker pid=69707) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=69707) End_Portfolio_Value: 4860617.5
(RolloutWorker pid=69707) Annual Return: 5.28 %
(RolloutWorker pid=69707) Worker ID: ABT.US Current Stock Exposure: 0
(RolloutWorker pid=69707) Worker ID: AMGN.US Current Stock Exposure: 54822
(RolloutWorker pid=69707) Worker ID: BDX.US Current Stock Exposure: 7567
(RolloutWorker pid=69707) Worker ID: BMY.US Current Stock Exposure: 75302
(RolloutWorker pid=69707) Worker ID: HUM.US Current Stock Exposure: 345350
(RolloutWorker pid=69707) Worker ID: JNJ.US Current Stock Exposure: 711953
(RolloutWorker pid=69707) Worker ID: LLY.US Current Stock Exposure: 288738
(RolloutWorker pid=69707) Worker ID: MDT.US Current Stock Exposur

(RolloutWorker pid=69703) 2023-09-06 22:59:26,233	WARNING env_runner_v2.py:155 -- More than 88270 observations in 6305 env steps for episode 441432604616471683 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.
(AIRA2C pid=69670) 2023-09-06 23:02:38,545	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration

(RolloutWorker pid=69703) =========HRL is done============= [repeated 5x across cluster]
(RolloutWorker pid=69703) HRL is done [repeated 5x across cluster]
(RolloutWorker pid=69703) day: 6304, episode: 3 [repeated 5x across cluster]
(RolloutWorker pid=69703) Total Cash Transfers: 65 [repeated 5x across cluster]
(RolloutWorker pid=69703) total_portfolio_trades: 47155.0 [repeated 5x across cluster]
(RolloutWorker pid=69703) Beginn_Portfolio_Value: 2000000 [repeated 5x across cluster]
(RolloutWorker pid=69703) End_Portfolio_Value: 9417435.0 [repeated 5x across cluster]
(RolloutWorker pid=69703) Annual Return: 9.38 % [repeated 5x across cluster]
(RolloutWorker pid=69703) Worker ID: ABT.US Current Stock Exposure: 0 [repeated 5x across cluster]
(RolloutWorker pid=69703) Worker ID: AMGN.US Current Stock Exposure: 1674876 [repeated 5x across cluster]
(RolloutWorker pid=69703) Worker ID: BDX.US Current Stock Exposure: 612802 [repeated 5x across cluster]
(RolloutWorker pid=69703) Worker ID: BMY.

(AIRA2C pid=69670) 2023-09-06 23:06:13,744	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').


...

(RolloutWorker pid=69704) =========HRL is done=============
(RolloutWorker pid=69704) HRL is done
(RolloutWorker pid=69704) day: 6304, episode: 6
(RolloutWorker pid=69704) Total Cash Transfers: 45
(RolloutWorker pid=69704) total_portfolio_trades: 36186.0
(RolloutWorker pid=69704) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=69704) End_Portfolio_Value: 12264601.0
(RolloutWorker pid=69704) Annual Return: 11.07 %
(RolloutWorker pid=69704) Worker ID: ABT.US Current Stock Exposure: 687968
(RolloutWorker pid=69704) Worker ID: AMGN.US Current Stock Exposure: 71163
(RolloutWorker pid=69704) Worker ID: BDX.US Current Stock Exposure: 52092
(RolloutWorker pid=69704) Worker ID: BMY.US Current Stock Exposure: 311914
(RolloutWorker pid=69704) Worker ID: HUM.US Current Stock Exposure: 852972
(RolloutWorker pid=69704) Worker ID: JNJ.US Current Stock Exposure: 1257533
(RolloutWorker pid=69704) Worker ID: LLY.US Current Stock Exposure: 566435
(RolloutWorker pid=69704) Worker ID: MDT.US Current Sto

(AIRA2C pid=69670) 2023-09-06 23:17:13,150	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').


(RolloutWorker pid=69704) =========HRL is done============= [repeated 4x across cluster]
(RolloutWorker pid=69704) HRL is done [repeated 4x across cluster]
(RolloutWorker pid=69704) day: 6304, episode: 7 [repeated 4x across cluster]
(RolloutWorker pid=69704) Total Cash Transfers: 42 [repeated 4x across cluster]
(RolloutWorker pid=69704) total_portfolio_trades: 36248.0 [repeated 4x across cluster]
(RolloutWorker pid=69704) Beginn_Portfolio_Value: 2000000 [repeated 4x across cluster]
(RolloutWorker pid=69704) End_Portfolio_Value: 14436736.0 [repeated 4x across cluster]
(RolloutWorker pid=69704) Annual Return: 12.12 % [repeated 4x across cluster]
(RolloutWorker pid=69704) Worker ID: ABT.US Current Stock Exposure: 92829 [repeated 4x across cluster]
(RolloutWorker pid=69704) Worker ID: AMGN.US Current Stock Exposure: 3057962 [repeated 4x across cluster]
(RolloutWorker pid=69704) Worker ID: BDX.US Current Stock Exposure: 560508 [repeated 4x across cluster]
(RolloutWorker pid=69704) Worker ID

(AIRA2C pid=69670) 2023-09-06 23:20:57,272	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').


(RolloutWorker pid=69704) =========HRL is done=============
(RolloutWorker pid=69704) HRL is done
(RolloutWorker pid=69704) day: 6304, episode: 8
(RolloutWorker pid=69704) Total Cash Transfers: 43
(RolloutWorker pid=69704) total_portfolio_trades: 36300.0
(RolloutWorker pid=69704) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=69704) End_Portfolio_Value: 18918238.0
(RolloutWorker pid=69704) Annual Return: 13.89 %
(RolloutWorker pid=69704) Worker ID: ABT.US Current Stock Exposure: 451647
(RolloutWorker pid=69704) Worker ID: AMGN.US Current Stock Exposure: 1793158
(RolloutWorker pid=69704) Worker ID: BDX.US Current Stock Exposure: 2186966
(RolloutWorker pid=69704) Worker ID: BMY.US Current Stock Exposure: 1413645
(RolloutWorker pid=69704) Worker ID: HUM.US Current Stock Exposure: 1495154
(RolloutWorker pid=69704) Worker ID: JNJ.US Current Stock Exposure: 953943
(RolloutWorker pid=69704) Worker ID: LLY.US Current Stock Exposure: 1533210
(RolloutWorker pid=69704) Worker ID: MDT.US Curre

In [ ]:
train_df.columns

Index(['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14'],
      dtype='object')

In [ ]:
# worker_df = train_df[train_df["tic"] == "PFE.US"]

# config={"df": worker_df}
        
# env = Worker(env_config=config)
# check_env(env)



# n_iterations = 500

# state = env.reset()

# for _ in range(n_iterations):

#     action = env.action_space.sample()

#     obs, reward, done, truncated, info= env.step(action)

#     # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
#     if done:
#         print("Episode finished!")
#         state = env.reset()
#     else:
#         state = obs

In [ ]:
manager_config = {
    "df": train_df,

}
config={



        "manager_config": manager_config
        }
env = HRL(env_config=config)
check_env(env)

In [3]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()

2023-09-06 23:22:30,851	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-06 23:22:30,855	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-09-06 23:22:30,895	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-06 23:22:30,941	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviou

== Status ==
Current time: 2023-09-06 23:22:36 (running for 00:00:05.97)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/autobatch/A2C
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+-----------------+----------+-----------------------+----------------------+--------------------+
| Trial name          | status   | loc   |   entropy_coeff |    gamma |   num_envs_per_worker |   sgd_minibatch_size |   train_batch_size |
|---------------------+----------+-------+-----------------+----------+-----------------------+----------------------+--------------------|
| A2C_hrl_7cd10_00000 | PENDING  |       |            0.1  | 0.99721  |                     1 |                  100 |                200 |
| A2C_hrl_7cd10_00001 | PENDING  |       |            1    | 0.929554 |                     2 |                  100 |                200 |
| A2C_hrl_7cd10_00002 | PENDING  |

(pid=74250) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2023-09-06 23:22:47 (running for 00:00:16.06)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/autobatch/A2C
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+-----------------+----------+-----------------------+----------------------+--------------------+
| Trial name          | status   | loc   |   entropy_coeff |    gamma |   num_envs_per_worker |   sgd_minibatch_size |   train_batch_size |
|---------------------+----------+-------+-----------------+----------+-----------------------+----------------------+--------------------|
| A2C_hrl_7cd10_00000 | PENDING  |       |            0.1  | 0.99721  |                     1 |                  100 |                200 |
| A2C_hrl_7cd10_00001 | PENDING  |       |            1    | 0.929554 |                     2 |                  100 |                200 |
| A2C_hrl_7cd10_00002 | PENDING 

(A2C pid=74251) 2023-09-06 23:22:48,494	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=74251) 2023-09-06 23:22:48,494	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=74251) 2023-09-06 23:22:48,494	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus_per_trial not supported.


...

== Status ==
Current time: 2023-09-06 23:32:15 (running for 00:09:44.82)
PopulationBasedTraining: 8 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/autobatch/A2C
Number of trials: 10/10 (7 PAUSED, 1 PENDING, 2 RUNNING)
+---------------------+----------+-----------------+-----------------+----------+-----------------------+----------------------+--------------------+--------+------------------+------+----------+----------------------+----------------------+--------------------+
| Trial name          | status   | loc             |   entropy_coeff |    gamma |   num_envs_per_worker |   sgd_minibatch_size |   train_batch_size |   iter |   total time (s) |   ts |   reward |   episode_reward_max |   episode_reward_min |   episode_len_mean |
|---------------------+----------+-----------------+-----------------+----------+-----------------------+----------------------+--------------------+--------+------------------+------+--

(pid=77044) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2023-09-06 23:32:29 (running for 00:09:58.33)
PopulationBasedTraining: 8 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/autobatch/A2C
Number of trials: 10/10 (7 PAUSED, 1 PENDING, 2 RUNNING)
+---------------------+----------+-----------------+-----------------+----------+-----------------------+----------------------+--------------------+--------+------------------+------+----------+----------------------+----------------------+--------------------+
| Trial name          | status   | loc             |   entropy_coeff |    gamma |   num_envs_per_worker |   sgd_minibatch_size |   train_batch_size |   iter |   total time (s) |   ts |   reward |   episode_reward_max |   episode_reward_min |   episode_len_mean |
|---------------------+----------+-----------------+-----------------+----------+-----------------------+----------------------+--------------------+--------+------------------+------+--

(RolloutWorker pid=77044) 2023-09-06 23:32:30,888	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=77044) 2023-09-06 23:32:30,888	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.tf_modelv2.TFModelV2` has been deprecated. Use `ray.rllib.core.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=77044) 2023-09-06 23:32:30,896	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.misc.normc_initializer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=77044) 2023-09-06 23:32:30,966	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=77044) 2023-09-06 23:32:30,966	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an

TuneError: The Ray Tune run failed. Please inspect the previous error messages for a cause. After fixing the issue, you can restart the run from scratch or continue this run. To continue this run, you can use `tuner = Tuner.restore("/Users/floriankockler/rayresults/autobatch/A2C", trainable=...)`.

In [ ]:

manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)°
env.observation_space.sample()

In [7]:
 
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hierarch_env", env_creator)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 

first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 
manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }


trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5},local_dir="/Users/floriankockler/rayresults/training"),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),
 
    algorithm=a2c.A2C,
    config={
        "multiagent": {
            "policies": {
                "worker_policy": worker_policy_spec,
                "manager_policy": manager_policy_spec,
            },
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env": "hierarch_env",
        "env_config": hrl_config,
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

2023-09-06 13:24:31,032	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=92326) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=92340) DeprecationWarning

TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = RLTrainer.restore("/Users/floriankockler/rayresults/training/AIRA2C_2023-09-06_13-24-30")`.
To start a new run that will retry on training failures, set `air.RunConfig(failure_config=air.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.

In [ ]:
# worker_config =   {
#         "cash_initial": 1000000, 
#         "trading_cost": 0.001, 
#         "tech_indicator_list": indicators,
#         "print_verbosity": 1,
#         "initial_shares_held": 0,
#         "df": df,  }


# env = Worker(env_config=worker_config)
# check_env(env)

In [ ]:


worker_config =   {
        "cash_initial": 1000000, 
        "trading_cost": 0.001, 
        "tech_indicator_list": indicators,
        "print_verbosity": 1,
        "initial_shares_held": 0,
        "df": train_df,  }


env = Worker(env_config=worker_config)

n_iterations = 30

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [4]:
import random

from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    # Assuming this is your environment
    return Worker(env_config)

register_env("Single_Stock", env_creator)


parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

worker_config =   {
        "cash_initial": 1000000, 
        "trading_cost": 0.001, 
        "tech_indicator_list": indicators,
        "print_verbosity": 1,
        "initial_shares_held": 0,
        "df": train_df,  
        }

def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    "rollout_fragment_length": [75],
    "train_batch_size": lambda: random.randint(1000, 50000),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),
    "train_batch_size": tune.choice([200, 400, 600]),
}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "Single_Stock",
        "env_config": worker_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 5,  # 1 for training + 4 for sampling
        "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/tuning3", progress_reporter=reporter),
)
results = tuner.fit()

2023-09-06 23:50:44,830	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2023-09-06 23:50:47 (running for 00:00:02.93)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2023-09-06 23:50:52 (running for 00:00:08.03)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (10 PENDING)




(pid=80798) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=80798) 2023-09-06 23:50:53,721	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=80798) 2023-09-06 23:50:53,721	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=80798) 2023-09-06 23:50:53,721	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus not supported.


== Status ==
Current time: 2023-09-06 23:50:57 (running for 00:00:13.13)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2023-09-06 23:51:03 (running for 00:00:18.16)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (10 PENDING)




(pid=80814) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=80813) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=80798) 2023-09-06 23:51:06,623	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=80812, ip=127.0.0.1, actor_id=4d937945e844fb743679ca2501000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x17ccfba00>)
(A2C pid=80798)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 209, in check_gym_environments
(A2C pid=80798)     raise ValueError(
(A2C pid=80798) ValueError: The observation collected from env.reset() was not contained within your env's observation space. It is possible that there was a type mismatch, or that one of the sub-observations 

...

== Status ==
Current time: 2023-09-06 23:51:39 (running for 00:00:54.64)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (1 ERROR, 9 PENDING)
Number of errored trials: 1
+------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   |   # failures | error file                                                                                                                                            |
|------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------|
| A2C_Single_Stock_6e71a_00000 |            1 | /Users/floriankockler/rayresults/tuning3/A2C/A2C_S

(pid=80908) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2023-09-06 23:51:44 (running for 00:00:59.69)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (1 ERROR, 9 PENDING)
Number of errored trials: 1
+------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   |   # failures | error file                                                                                                                                            |
|------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------|
| A2C_Single_Stock_6e71a_00000 |            1 | /Users/floriankockler/rayresults/tuning3/A2C/A2C_S

(A2C pid=80874) 2023-09-06 23:51:44,819	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=80908, ip=127.0.0.1, actor_id=6771037b223f8c60f191019b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x177bdfa60>)
(A2C pid=80874)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 209, in check_gym_environments
(A2C pid=80874)     raise ValueError(
(A2C pid=80874) ValueError: The observation collected from env.reset() was not contained within your env's observation space. It is possible that there was a type mismatch, or that one of the sub-observations was out of bounds:
(A2C pid=80874)  path: 'current_price'
(A2C pid=80874)  (sub-)obs: [[ 0.8285  0.192   2.1572  2.0991 15.9956  1.489   2.3942  0.4024  1.1337
(A2C pid=80874)    0.7178  0.4057  2.7255  0.2499]] (float32)
(A2C pid

== Status ==
Current time: 2023-09-06 23:51:49 (running for 00:01:04.83)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 6.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuning3/A2C
Number of trials: 10/10 (2 ERROR, 8 PENDING)
Number of errored trials: 2
+------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   |   # failures | error file                                                                                                                                            |
|------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------|
| A2C_Single_Stock_6e71a_00000 |            1 | /Users/floriankockler/rayresults/tuning3/A2C/A2C_S